In [1]:
import pandas as pd
import numpy as np
from fbprophet import Prophet

In [4]:
df = pd.read_csv('data/beijing_201802_201803_aq.csv')
df['utc_time'] = pd.to_datetime(df['utc_time'])

In [11]:
pm25_df = df[['utc_time', 'PM2.5']]
pm25_df.columns = ['ds', 'y']

In [12]:
pm25_df = pm25_df.dropna()

In [13]:
m = Prophet()
m.fit(pm25_df);

INFO:fbprophet.forecaster:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
/anaconda3/envs/python36/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


In [18]:
future = m.make_future_dataframe(periods=60)
future

,ds
0,2018-01-31 16:00:00
1,2018-01-31 16:00:00
2,2018-01-31 16:00:00
3,2018-01-31 16:00:00
4,2018-01-31 16:00:00
5,2018-01-31 16:00:00
6,2018-01-31 16:00:00
7,2018-01-31 16:00:00
8,2018-01-31 16:00:00
9,2018-01-31 16:00:00


In [19]:
forecast = m.predict(future)

In [37]:
res_df = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
res_df = res_df[res_df.ds >= '2018-03-15']

In [38]:
res_df

,ds,yhat,yhat_lower,yhat_upper
33288,2018-03-15 00:00:00,95.656898,35.347088,160.679085
33289,2018-03-15 00:00:00,95.656898,39.088275,154.327855
33290,2018-03-15 00:00:00,95.656898,34.006131,155.329244
33291,2018-03-15 00:00:00,95.656898,35.600970,157.032595
33292,2018-03-15 00:00:00,95.656898,31.068044,156.999313
33293,2018-03-15 00:00:00,95.656898,32.019325,156.990109
33294,2018-03-15 00:00:00,95.656898,29.500734,155.858310
33295,2018-03-15 00:00:00,95.656898,32.839267,163.283774
33296,2018-03-15 00:00:00,95.656898,37.314494,153.594163
33297,2018-03-15 00:00:00,95.656898,39.507240,158.634807


In [39]:
pm25_df[pm25_df['ds'] >= '2018-03-15']

,ds,y
23724,2018-03-15 00:00:00,10.0
23725,2018-03-15 01:00:00,4.0
23726,2018-03-15 02:00:00,5.0
23727,2018-03-15 03:00:00,7.0
23728,2018-03-15 04:00:00,3.0
23729,2018-03-15 05:00:00,10.0
23730,2018-03-15 06:00:00,4.0
23731,2018-03-15 07:00:00,7.0
23732,2018-03-15 08:00:00,3.0
23733,2018-03-15 09:00:00,5.0


In [45]:
Prophet??